In [1]:
import numpy as np                                  # type: ignore
from skimage.measure import label, regionprops      # type: ignore
from skimage.io import imread, imshow               # type: ignore
from skimage.filters import threshold_yen           # type: ignore
import glob
from dataclasses import dataclass
import napari
import os
import time as clock
import myfunctions as mf
from tqdm import tqdm

In [2]:
exp = mf.exp_list()[20]
print(exp)
first_slice = 100
last_slice = 100
OS = 'Windows'

sequence = mf.read_4Dsequence(exp, first_slice=first_slice, last_slice=last_slice, OS=OS, skip180=True)

viewer = napari.Viewer()
images = [viewer.add_image(sequence, name='Volume')]
viewer.dims.current_step = (0, 0)

VCT5A_FT_H_Exp5
